In [2]:
# Anchor Portfolio 
import numpy as np 


def equal_weighted_portfolio(num_assets):
    """
    Create an equal-weighted anchor portfolio.

    Parameters:
    - num_assets (int): Number of assets in the portfolio.

    Returns:
    - np.array: Equal weights for all assets.
    """
    return np.ones(num_assets) / num_assets

# Example: 3 assets
num_assets = 3
anchor_eq = equal_weighted_portfolio(num_assets)
print("Equal-Weighted Portfolio:", anchor_eq)


def volatility_weighted_portfolio(volatilities):
    """
    Create a volatility-weighted anchor portfolio.

    Parameters:
    - volatilities (np.array): Standard deviations (volatilities) of the assets.

    Returns:
    - np.array: Weights inversely proportional to volatilities.
    """
    inverse_vol = 1 / volatilities
    return inverse_vol / np.sum(inverse_vol)

# Example: Volatilities for 3 assets
volatilities = np.array([0.2, 0.3, 0.5])
anchor_vol = volatility_weighted_portfolio(volatilities)
print("Volatility-Weighted Portfolio:", anchor_vol)



Equal-Weighted Portfolio: [0.33333333 0.33333333 0.33333333]
Volatility-Weighted Portfolio: [0.48387097 0.32258065 0.19354839]


Anchored EPO Process: tune the shrinkage parameter w to balance between the expected returns and the anchor portfolio 
- Higher w =  more weight on the anchor portfolio, making the optimization more conservative and less sensitive to noisy expected returns.
- Also more stable with a stronger anchor

** Done through the balance of covariacnce and expected returns 

- Risk Aversion Parameter: trade off between risk and returns (influence portfolio aggresiveness) - risk vs returns
- Signal Confidence - more trust in the signal or more risk control

- metric: sharpe ratio
𝑤
w places more weight on the signal (expected returns), allowing for a more aggressive tilt toward assets with higher predicted returns.


In [ ]:
Covariance: 
- quantifies how asset returns move relative to each other 
calculates portfolio variance. lower covariance = lower portfolio risk through diversification 


In [6]:
import numpy as np

def optimize_shrinkage(expected_returns, cov_matrix, anchor_portfolio, risk_aversion, w_range):
    """
    Optimize shrinkage parameter w for Anchored EPO.

    Parameters:
    - expected_returns (np.array): Vector of expected returns (signal).
    - cov_matrix (np.array): Covariance matrix of asset returns.
    - anchor_portfolio (np.array): Anchor portfolio weights.
    - risk_aversion (float): Risk aversion parameter.
    - w_range (np.array): Range of shrinkage values to test.

    Returns:
    - optimal_w (float): Best shrinkage parameter w.
    - optimal_weights (np.array): Portfolio weights with optimal shrinkage.
    """
    best_sharpe = -np.inf
    optimal_w = None
    optimal_weights = None

    for w in w_range:
        # Compute shrinkage
        shrunk_cov_matrix = (1 - w) * cov_matrix + w * np.diag(np.diag(cov_matrix))
        shrunk_expected_returns = (1 - w) * expected_returns + w * np.dot(np.diag(np.diag(cov_matrix)), anchor_portfolio)

        # Compute portfolio weights
        weights = np.linalg.solve(shrunk_cov_matrix, shrunk_expected_returns) / risk_aversion

        # Compute portfolio performance (Sharpe ratio)
        portfolio_return = np.dot(weights, expected_returns)
        portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        sharpe_ratio = portfolio_return / portfolio_risk

        # Update best parameters
        if sharpe_ratio > best_sharpe:
            best_sharpe = sharpe_ratio
            optimal_w = w
            optimal_weights = weights

    return optimal_w, optimal_weights


# Example Inputs
expected_returns = np.array([0.08, 0.12, 0.10])  # Signal
cov_matrix = np.array([[0.04, 0.01, 0.02], [0.01, 0.09, 0.03], [0.02, 0.03, 0.16]])
anchor_portfolio = np.array([0.3, 0.4, 0.3])  # Predefined anchor portfolio
risk_aversion = 3.0
w_range = np.linspace(0, 1, 50)  # Shrinkage values to test

# Optimize shrinkage
optimal_w, optimal_weights = optimize_shrinkage(expected_returns, cov_matrix, anchor_portfolio, risk_aversion, w_range)

# Display Results
print("Optimal Shrinkage (w):", optimal_w)
print("Optimal Portfolio Weights:", optimal_weights)



gamma_values = [1, 2, 3, 5, 10]
for gamma in gamma_values:
    weights = np.linalg.solve(cov_matrix, expected_returns) / gamma
    portfolio_return = np.dot(weights, expected_returns)
    portfolio_risk = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
    sharpe_ratio = portfolio_return / portfolio_risk
    print(f"Gamma: {gamma}, Sharpe Ratio: {sharpe_ratio}")

tau_values = np.linspace(0, 1, 10)
for tau in tau_values:
    shrunk_returns = tau * expected_returns + (1 - tau) * np.dot(np.diag(np.diag(cov_matrix)), anchor_portfolio)
    weights = np.linalg.solve(cov_matrix, shrunk_returns) / risk_aversion
    print(f"Tau: {tau}, Weights: {weights}")


Optimal Shrinkage (w): 0.0
Optimal Portfolio Weights: [0.54       0.36       0.07333333]
Gamma: 1, Sharpe Ratio: 0.530282943342514
Gamma: 2, Sharpe Ratio: 0.530282943342514
Gamma: 3, Sharpe Ratio: 0.530282943342514
Gamma: 5, Sharpe Ratio: 0.5302829433425141
Gamma: 10, Sharpe Ratio: 0.5302829433425141
Tau: 0.0, Weights: [0.036 0.104 0.076]
Tau: 0.1111111111111111, Weights: [0.092      0.13244444 0.0757037 ]
Tau: 0.2222222222222222, Weights: [0.148      0.16088889 0.07540741]
Tau: 0.3333333333333333, Weights: [0.204      0.18933333 0.07511111]
Tau: 0.4444444444444444, Weights: [0.26       0.21777778 0.07481481]
Tau: 0.5555555555555556, Weights: [0.316      0.24622222 0.07451852]
Tau: 0.6666666666666666, Weights: [0.372      0.27466667 0.07422222]
Tau: 0.7777777777777777, Weights: [0.428      0.30311111 0.07392593]
Tau: 0.8888888888888888, Weights: [0.484      0.33155556 0.07362963]
Tau: 1.0, Weights: [0.54       0.36       0.07333333]
